<a href="https://colab.research.google.com/github/s00d/roop_face_replacement/blob/main/roop_face_replacement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 1) Проверяем видеокарту/Check graphics card
!nvidia-smi

In [ ]:
#@title 2) Клонируем репозиторий с github/Clone github repo
!git clone https://github.com/s0md3v/roop

In [ ]:
#@title 3) Устанавливаем зависимости/Install dependencies
%cd /content/roop
!python -m pip install --upgrade pip
!pip install -r requirements.txt
!pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu118

In [ ]:
#@title 4) Загружаем фото с лицом/Upload photo with face
%cd /content/roop
!mkdir files
%cd /content/roop/files
from google.colab import files
uploaded = files.upload()
for i in uploaded.keys():
  face_raw = "/content/roop/files/" + i
  break 
!mv "$face_raw" /content/roop/files/face.jpg

In [ ]:
#@title 5) Загружаем видео/Upload video
%cd /content/roop/files
from google.colab import files
uploaded = files.upload()
for i in uploaded.keys():
  video_raw = "/content/roop/files/" + i
  break 
!mv "$video_raw" /content/roop/files/video.mp4

In [ ]:
#@title 6) Скачиваем inswapper_128.onnx/Download inswapper_128.onnx
%cd /content/roop/
import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output
HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
)
#@title MEGA public link download
URL = "https://mega.nz/file/9l8mGDJA#FnPxHwpdhDovDo6OvbQjhHd2nDAk8_iVEgo3mpHLG6U"
OUTPUT_PATH = "/content/roop/inswapper_128.onnx" 
if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        


transfare()

Если 6 пункт не работает, то: 

1.   Скачиваем файл https://drive.google.com/file/d/1eu60OrRtn4WhKrzM4mQv4F3rIuyUXqfl/view
2.   Заливаем его на Гугл Диск, открываем доступ по ссылке, копируем её и переходим по ней же
3.   Жмем "скачать", когда нам напишет "Не удалось проверить файл на наличие вирусов" копируем ссылку из адресной строки без "&export=download" (должно получиться что-то типа "https://drive.google.com/u/0/uc?id=1eu60OrRtn4WhKrzM4mQv4F3rIuyUXqfl" (можно попробовать и эту ссылку вставить, но не факт, что сработает))
4.   Вставляем ссылку в следующую ячейку и выполняем её


In [ ]:
#@title 6.1) Скачиваем inswapper_128.onnx с Гугл Диска/Download inswapper_128.onnx from Google Drive
#@markdown ЗАПУСКАТЬ ТОЛЬКО ЕСЛИ НЕ РАБОТАЕТ 6 ПУНКТ: 

%cd /content/roop/
import gdown
#@markdown Ссылка на файл: 
url = "https://drive.google.com/u/0/uc?id=1eu60OrRtn4WhKrzM4mQv4F3rIuyUXqfl" #@param {type:"string"}
gdown.download(url, "/content/roop/inswapper_128.onnx" , quiet=False)

In [ ]:
#@title 7) Заменяем лицо/Face replace
%cd /content/roop/
!touch /content/output.mp4
!python run.py -f /content/roop/files/face.jpg -t /content/roop/files/video.mp4 -o /content/output.mp4 --gpu 
from google.colab import files
files.download('/content/output.mp4')